In [1]:
import timeit

import pandas as pd
import mysql.connector

# Carregamento

In [2]:
df_viagem = pd.read_csv('./dataset/gerados/viagem.csv')
df_passagem = pd.read_csv('./dataset/gerados/passagem.csv')
df_pagamento = pd.read_csv('./dataset/gerados/pagamento.csv')
df_cpf = pd.read_csv('./dataset/gerados/cpf.csv')
df_puc = pd.read_csv('./dataset/gerados/puc.csv')
df_paises = pd.read_csv('./dataset/gerados/paises.csv')
df_uf = pd.read_csv('./dataset/gerados/uf.csv')
df_cidades = pd.read_csv('./dataset/gerados/cidades.csv')
df_pessoa = pd.read_csv('./dataset/gerados/pessoa.csv')
df_cargo = pd.read_csv('./dataset/gerados/cargo.csv')
df_funcao = pd.read_csv('./dataset/gerados/funcao.csv')
df_ministerios = pd.read_csv('./dataset/gerados/ministerio.csv')
df_meio = pd.read_csv('./dataset/gerados/meio.csv')

In [3]:
df_viagem = df_viagem.drop_duplicates(subset=['id_viagem'])

# ETL

In [30]:
cnx = mysql.connector.connect(user='root', password='root',
                              host='localhost',
                              database='VIAGENS_DB')

In [31]:
cursor = cnx.cursor()

In [6]:
# Para 2 campos: ID(padrao) e NOME
dict_colunas = {'PAIS': df_paises,
                'CIDADE': df_cidades, 
                'UF': df_uf, 
                'MEIO_TRANSPORTE': df_meio, 
                'CARGO': df_cargo,
               }

In [7]:
%%time
for name, df in dict_colunas.items():
    print(name)
    for _, row in enumerate(df.itertuples()):
        query = f"INSERT INTO {name} "\
                 f"(NOME)" \
                 f"VALUES ({row.nome!r})"
        cursor.execute(query)

PAIS
CIDADE
UF
MEIO_TRANSPORTE
CARGO
CPU times: user 38.6 ms, sys: 28.3 ms, total: 66.9 ms
Wall time: 177 ms


In [8]:
cnx.commit()

In [9]:
%%time
# Ministerio
for _, row in enumerate(df_ministerios.itertuples()):
    query = f"INSERT INTO MINISTERIO"\
             f"(ID_MINISTERIO, NOME)" \
             f"VALUES ({row.id_ministerio!r}, {row.nome!r})"
    cursor.execute(query)

CPU times: user 1.92 ms, sys: 296 µs, total: 2.22 ms
Wall time: 3.92 ms


In [10]:
cnx.commit()

In [11]:
%%time
# Pessoa
for _, row in enumerate(df_pessoa.itertuples()):
    query = f"INSERT INTO PESSOA"\
             f"(NOME, CPF)" \
             f"VALUES ({row.nome!r}, {row.cpf!r})"
    cursor.execute(query)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.53 µs


In [12]:
cnx.commit()

In [14]:
%%time
# Funcao
for _, row in enumerate(df_funcao.itertuples()):
    query = f"INSERT INTO FUNCAO"\
             f"(NOME, DESCRICAO)" \
             f"VALUES ({row.nome!r}, {row.descricao!r})"
    cursor.execute(query)

CPU times: user 5.71 ms, sys: 0 ns, total: 5.71 ms
Wall time: 12.8 ms


In [15]:
cnx.commit()

In [16]:
%%time
# CPF
for _, row in enumerate(df_cpf.itertuples()):
    query = f"INSERT INTO PESSOA_CARGO_FUNCAO"\
             f"(ID_PESSOA, ID_FUNCAO, ID_CARGO)" \
             f"VALUES ({row.id_pessoa!r}, {row.id_funcao!r}, {row.id_cargo!r})"
    cursor.execute(query)

CPU times: user 2.85 s, sys: 1.19 s, total: 4.04 s
Wall time: 12.1 s


In [17]:
cnx.commit()

In [18]:
%%time
# PUC
for _, row in enumerate(df_puc.itertuples()):
    query = f"INSERT INTO PAIS_CIDADE_UF"\
             f"(ID_PAIS, ID_UF, ID_CIDADE)" \
             f"VALUES ({row.id_pais!r}, {row.id_uf!r}, {row.id_cidade!r})"
    cursor.execute(query)

CPU times: user 17.3 ms, sys: 8.69 ms, total: 26 ms
Wall time: 73.8 ms


In [19]:
cnx.commit()

In [20]:
%%time
# VIAGEM
for _, row in enumerate(df_viagem.itertuples()):
    query = f"INSERT INTO VIAGEM"\
             f"(ID_VIAGEM, IS_REALIZADO, IS_URGENTE, DATA_INICIO, DATA_FIM, ID_MINISTERIO, ID_PESSOA_CARGO_FUNCAO)" \
             f"VALUES ({row.id_viagem!r}, {row.is_realizado!r}, {row.is_urgente!r}, {row.data_inicio!r}, {row.data_fim!r}, {row.id_ministerio!r}, {row.id_cpf!r})"
    cursor.execute(query)

CPU times: user 8.71 s, sys: 3.64 s, total: 12.4 s
Wall time: 37.6 s


In [21]:
cnx.commit()

In [23]:
%%time
# PASSAGEM
for _, row in enumerate(df_passagem.itertuples()):
    query = f"INSERT INTO PASSAGEM"\
             f"(ID_VIAGEM, ID_MEIO_TRANSPORTE, ID_PAIS_CIDADE_UF_ORIGEM_IDA, ID_PAIS_CIDADE_UF_DESTINO_IDA, ID_PAIS_CIDADE_UF_ORIGEM_VOLTA, ID_PAIS_CIDADE_UF_DESTINO_VOLTA)" \
             f"VALUES ({row.id_viagem!r}, {row.id_meio!r}, {row.id_puc_origem_ida!r}, {row.id_puc_destino_ida!r}, {row.id_puc_origem_volta!r}, {row.id_puc_destino_volta!r})"
    cursor.execute(query)

CPU times: user 2.75 s, sys: 1.18 s, total: 3.93 s
Wall time: 12.5 s


In [24]:
cnx.commit()

In [32]:
%%time
# Pagamento
for _, row in enumerate(df_pagamento.itertuples()):
    print(_, end='\r')
    query = f"INSERT INTO PAGAMENTO"\
             f"(VALOR, TAXA, ID_PASSAGEM)" \
             f"VALUES ({row.valor!r}, {row.taxa!r}, {row.id_passagem!r})"
    cursor.execute(query)

CPU times: user 43.3 s, sys: 6.4 s, total: 49.7 s
Wall time: 3h 10min 49s


In [33]:
cnx.commit()

In [28]:
cursor.reset()

In [29]:
cnx.close()